In [1]:
import geopandas as gpd
from shapely.geometry import LineString
import rasterio
import numpy as np
from rasterio.sample import sample_gen
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
from rasterio.sample import sample_gen
from pathlib import Path



In [2]:
ice_gdf = gpd.read_parquet("icesat2_region_orthometric.parquet")

In [3]:
ice_gdf = ice_gdf.to_crs("EPSG:4326")


In [4]:
coords = [(geom.x, geom.y) for geom in ice_gdf.geometry]


In [5]:
print("ICESat CRS:", ice_gdf.crs)


ICESat CRS: EPSG:4326


In [6]:
dem_name_map = {
    "tan_dem": "TanDEM-X",
    "srtm_dem": "SRTM",
    "fab_dem": "FABDEM",
    "copernicus_dem": "Copernicus DEM",
    "nasa_dem": "NASADEM",
    "alos_dem": "ALOS",
    "aster_dem": "ASTER"}

,name,path,crs,width,height,resolution,bounds
0,alos_dem,data_clipped/alos/alos_dem.tif,EPSG:4326,1464,1555,"(0.0002777777777777778, 0.0002777777777777778)","(24.716944444444444, 47.693888888888885, 25.12..."
1,alos_dem_egg2015,data_clipped/alos/alos_dem_egg2015.tif,EPSG:4326,1464,1555,"(0.0002777777777777778, 0.0002777777777777778)","(24.716944444444444, 47.693888888888885, 25.12..."
2,alos_dem_ellip,data_clipped/alos/alos_dem_ellip.tif,EPSG:4326,1464,1555,"(0.0002777777777777778, 0.0002777777777777778)","(24.716944444444444, 47.693888888888885, 25.12..."
3,aster_dem,data_clipped/aster/aster_dem.tif,EPSG:4326,1465,1556,"(0.000277777777777778, 0.000277777777777778)","(24.716805555555545, 47.69375000000001, 25.123..."
4,aster_dem_egg2015,data_clipped/aster/aster_dem_egg2015.tif,EPSG:4326,1465,1556,"(0.000277777777777778, 0.000277777777777778)","(24.716805555555545, 47.69375000000001, 25.123..."
5,aster_dem_ellip,data_clipped/aster/aster_dem_ellip.tif,EPSG:4326,1465,1556,"(0.000277777777777778, 0.000277777777777778)","(24.716805555555545, 47.69375000000001, 25.123..."
6,copernicus_dsm_4326,data_clipped/copernicus/copernicus_dsm_4326.tif,EPSG:4326,1465,1556,"(0.0002777777777777778, 0.0002777777777777778)","(24.716805555555556, 47.69375, 25.12375, 48.12..."
7,copernicus_dеm_egg2015,data_clipped/copernicus/copernicus_dеm_egg2015...,EPSG:4326,1465,1556,"(0.0002777777777777778, 0.0002777777777777778)","(24.716805555555556, 47.69375, 25.12375, 48.12..."
8,copernicus_dеm_ellip,data_clipped/copernicus/copernicus_dеm_ellip.tif,EPSG:4326,1465,1556,"(0.0002777777777777778, 0.0002777777777777778)","(24.716805555555556, 47.69375, 25.12375, 48.12..."
9,fab_dem,data_clipped/fabdem/fab_dem.tif,EPSG:4326,1465,1556,"(0.0002777777777777778, 0.0002777777777777778)","(24.716805555555556, 47.69375, 25.12375, 48.12..."


In [8]:

dem_root = Path("data_clipped")
dem_paths = list(dem_root.rglob("*_egg2015.tif"))

# Витяг координат один раз
coords = [(geom.x, geom.y) for geom in ice_gdf.geometry]

# Основний цикл
for dem_path in dem_paths:
    name = dem_path.stem.replace("_egg2015", "")
    with rasterio.open(dem_path) as src:
        # Перевірка CRS
        if src.crs.to_epsg() not in [4326, 4979] or ice_gdf.crs.to_epsg() not in [4326, 4979]:
            raise ValueError(f"❌ CRS не сумісні: {name} vs ICESat")

        # Витяг DEM-висот
        dem_values = list(sample_gen(src, coords))
        col_dem = f"h_{name}"
        col_delta = f"delta_{name}"

        # Запис у GeoDataFrame
        ice_gdf[col_dem] = [val[0] if val else None for val in dem_values]
        ice_gdf[col_delta] = ice_gdf[col_dem] - ice_gdf["orthometric_height"]

        print(f"✅ {name}: додано {col_dem} та {col_delta}")

# Зберігаємо результат
ice_gdf.to_parquet("output/icesat2_dems_with_deltas.parquet", index=False)
print("📦 Файл збережено до output/icesat2_dems_with_deltas.parquet")


✅ alos_dem: додано h_alos_dem та delta_alos_dem
✅ aster_dem: додано h_aster_dem та delta_aster_dem
✅ copernicus_dеm: додано h_copernicus_dеm та delta_copernicus_dеm
✅ fab_dem: додано h_fab_dem та delta_fab_dem
✅ nasa_dem: додано h_nasa_dem та delta_nasa_dem
✅ srtm_dem: додано h_srtm_dem та delta_srtm_dem
✅ tan_dem: додано h_tan_dem та delta_tan_dem
📦 Файл збережено до output/icesat2_dems_with_deltas.parquet


In [1]:
# 1. Фільтрація надійних точок по Spot 3
filtered = ice_gdf[
    (ice_gdf["rgt"] == 396) &
    (ice_gdf["spot"] == 3) &
    (ice_gdf["atl03_cnf"] == 4) &
    (ice_gdf["atl08_class"].isin([1, 2, 3]))
].copy()

# 2. Додати дату
filtered["date"] = filtered.index.date
dates = sorted(filtered["date"].unique())

# 3. Візуалізація та обчислення по кожному проходу
plt.figure(figsize=(10, 6))

for date in dates:
    df = filtered[filtered["date"] == date].copy()
    if df.empty:
        continue

    df_sorted = df.sort_values("ph_index")

    if len(df_sorted) < 5:
        continue  # пропустити надто короткі проходи

    # Лінія треку та обчислення відстані
    line = LineString(df_sorted.geometry.tolist())
    df["distance_m"] = df.geometry.apply(lambda pt: line.project(pt))

    # Обчислення різниці
    df["delta"] = df["dem_height"] - df["orthometric_height"]
    mae = np.mean(np.abs(df["delta"]))
    rmse = np.sqrt(np.mean(df["delta"]**2))
    print(f"{date}: MAE = {mae:.2f} м, RMSE = {rmse:.2f} м")

    # Побудова графіків
    plt.plot(df["distance_m"], df["orthometric_height"], label=f"ICESat {date}")
    plt.plot(df["distance_m"], df["dem_height"], '--', label=f"DEM {date}")

# Оформлення графіка
plt.title("Профілі ортометричної висоти: ICESat-2 vs DEM (Spot 3)")
plt.xlabel("Відстань уздовж треку (м)")
plt.ylabel("Ортометрична висота (м)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'ice_gdf' is not defined

In [ ]:
min_points = 1000
good_dates = filtered.groupby('date').filter(lambda x: len(x) > min_points)['date'].unique()

In [ ]:
results = []

for date in dates:
    df = filtered[filtered['date'] == date].copy()

    if df.empty:
        continue

    # Сортування та обчислення дистанції вздовж треку
    df_sorted = df.sort_values("ph_index")
    line = LineString(df_sorted.geometry.tolist())
    df["distance_m"] = df.geometry.apply(lambda pt: line.project(pt))

    # Різниця DEM - ICESat
    df["delta"] = df["dem_height"] - df["orthometric_height"]

    # Статистика
    mae = np.mean(np.abs(df["delta"]))
    rmse = np.sqrt(np.mean(df["delta"]**2))
    results.append((str(date), len(df), mae, rmse))

# Сортуємо за RMSE
sorted_results = sorted(results, key=lambda x: x[3])  # сортування за RMSE

# Виводимо топ 5
for date, n, mae, rmse in sorted_results[:5]:
    print(f"{date}: {n} точок | MAE = {mae:.2f} м | RMSE = {rmse:.2f} м")


In [ ]:


def plot_profiles(
    df,
    group_column='date',
    title=None,
    figsize=(10, 6),
    dem_label="DEM",
    color_map_name='tab10',
    language="uk",  # "uk", "en", or "both"
    dem_name="DEM",
    save_path=None
):
    """
    Побудова порівняльних профілів ортометричних висот (ICESat-2 vs DEM) з автоматичним кольором.

    Parameters:
        df : pd.DataFrame
            Дані, які мають колонки: geometry, ph_index, orthometric_height, dem_height, group_column.
        group_column : str
            Колонка для групування (наприклад, 'date' або 'track_id').
        title : str
            Назва графіка.
        figsize : tuple
            Розмір графіка (default (10, 6)).
        dem_label : str
            Текст для підпису DEM у легенді (відображається один раз).
        color_map_name : str
            Назва палітри matplotlib (наприклад, 'tab10', 'Set1', 'Dark2', 'viridis').
        language : str
            Мова: "uk", "en" або "both".
        dem_name : str
            Назва DEM (TanDEM-X, FABDEM тощо), буде в заголовку.
        save_path : str
            Шлях для збереження графіка (наприклад, "output.png"). Якщо None — графік лише показується.
    """

    unique_groups = sorted(df[group_column].dropna().unique())
    cmap = plt.get_cmap(color_map_name)
    color_map = {group: cmap(i % cmap.N) for i, group in enumerate(unique_groups)}

    plt.style.use('default')  # білий фон
    plt.figure(figsize=figsize)

    dem_plotted = False

    for group in unique_groups:
        df_group = df[df[group_column] == group].copy()
        if df_group.empty:
            continue

        df_sorted = df_group.sort_values("ph_index")
        line = LineString(df_sorted.geometry.tolist())
        df_sorted["distance_m"] = df_sorted.geometry.apply(lambda pt: line.project(pt))

        plt.plot(df_sorted["distance_m"], df_sorted["orthometric_height"],
                 label=f"ICESat-2 {group}",
                 color=color_map[group],
                 linewidth=2)

        # DEM — один раз у легенді
        dem_legend = dem_label if not dem_plotted else "_nolegend_"
        plt.plot(df_sorted["distance_m"], df_sorted["dem_height"],
                 label=dem_legend,
                 color='gray',
                 linestyle='--',
                 linewidth=1.5)
        dem_plotted = True

    # Динамічна назва
    if not title:
        if language == "uk":
            title = f"Порівняння ортометричних висот ICESat-2 і DEM ({dem_name}) у системі EVRF2007"
            ylabel = "Ортометрична висота (м н.р.м.)"
            xlabel = "Відстань уздовж треку (м)"
        elif language == "en":
            title = f"Comparison of orthometric heights from ICESat-2 and {dem_name} DEM in the EVRF2007 system"
            ylabel = "Orthometric height (m a.s.l.)"
            xlabel = "Distance along track (m)"
        elif language == "both":
            title = f"Порівняння ортометричних висот (EVRF2007)\nComparison of orthometric heights from ICESat-2 and {dem_name} DEM"
            ylabel = "Ортометрична висота (м н.р.м.) / Orthometric height (m a.s.l.)"
            xlabel = "Відстань уздовж треку (м) / Distance along track (m)"
        else:
            title = "Orthometric height profiles"
            ylabel = "Height (m)"
            xlabel = "Distance (m)"

    plt.title(title, fontsize=14)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)

    # Впорядкування легенди: DEM завжди останній
    handles, labels = plt.gca().get_legend_handles_labels()
    dem_entries = [(h, l) for h, l in zip(handles, labels) if l == dem_label]
    other_entries = [(h, l) for h, l in zip(handles, labels) if l != dem_label]
    sorted_handles, sorted_labels = zip(*(other_entries + dem_entries))
    plt.legend(sorted_handles, sorted_labels,
               loc='center left', bbox_to_anchor=(1, 0.5),
               fontsize=10, frameon=False)

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    else:
        plt.show()

In [ ]:
top_dates = ["2025-05-07"]
top_dates_dt = [pd.to_datetime(d).date() for d in top_dates]
filtered_subset = filtered[filtered["date"].isin(top_dates_dt)]

plot_profiles(
    df=filtered_subset,
    dem_label="TanDEM-X DEM",
    dem_name="TanDEM-X",
    language="en",  # "uk", "en", або "both"
    save_path="output/icesat_vs_tandem_bilingual.png"
)